# 위키피디아 임베딩 pinecone DB구축

In [ ]:
# 'wiki' 인딕스 생성

In [5]:
from dotenv import load_dotenv
import os
load_dotenv()
PINECONE_API_KEY = os.getenv('PINECONE_API_KEY')

In [8]:
from pinecone import Pinecone, ServerlessSpec
pc = Pinecone(api_key=PINECONE_API_KEY)

In [9]:
from pinecone import Pinecone, ServerlessSpec
index_name = "wiki"

# Pinecone에 있는 모든 인덱스를 순회합니다.
for idx in pinecone_client.list_indexes():
    # 인덱스 이름이 "quickstart"와 일치하는 경우 해당 인덱스를 삭제합니다.
    if idx.name == index_name:
        pinecone_client.delete_index(idx.name)

In [10]:
pc.create_index(
    name=index_name,
    dimension=1536,  # 모델 차원
    metric="cosine",  # 모델 메트릭
    spec=ServerlessSpec(
        cloud="aws",
        region="us-east-1"
    )
)

{
    "name": "wiki",
    "metric": "cosine",
    "host": "wiki-2wxeig8.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 1536,
    "deletion_protection": "disabled",
    "tags": null
}

In [11]:
# 'wiki' 인덱스를 가져옵니다.
index = pc.Index(index_name)

index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {},
 'total_vector_count': 0,
 'vector_type': 'dense'}

In [ ]:
# data load

In [ ]:
# %pip install -qU datasets

Note: you may need to restart the kernel to use updated packages.


In [12]:
from datasets import load_dataset
data = load_dataset("wikimedia/wikipedia", "20231101.ko", split='train[:100]')

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`
Generating train split: 100%|██████████| 647897/647897 [00:01<00:00, 356701.79 examples/s]


In [13]:
data

Dataset({
    features: ['id', 'url', 'title', 'text'],
    num_rows: 100
})

In [14]:
data[0]

{'id': '5',
 'url': 'https://ko.wikipedia.org/wiki/%EC%A7%80%EB%AF%B8%20%EC%B9%B4%ED%84%B0',
 'title': '지미 카터',
 'text': '제임스 얼 카터 주니어(, 1924년 10월 1일~)는 민주당 출신 미국의 제39대 대통령(1977년~1981년)이다.\n\n생애\n\n어린 시절 \n지미 카터는 조지아주 섬터 카운티 플레인스 마을에서 태어났다.\n\n조지아 공과대학교를 졸업하였다. 그 후 해군에 들어가 전함·원자력·잠수함의 승무원으로 일하였다. 1953년 미국 해군 대위로 예편하였고 이후 땅콩·면화 등을 가꿔 많은 돈을 벌었다. 그의 별명이 "땅콩 농부" (Peanut Farmer)로 알려졌다.\n\n정계 입문 \n1962년 조지아주 상원 의원 선거에서 낙선하였으나, 그 선거가 부정선거 였음을 입증하게 되어 당선되고, 1966년 조지아 주지사 선거에 낙선하지만, 1970년 조지아 주지사 선거에서 당선됐다. 대통령이 되기 전 조지아주 상원의원을 두번 연임했으며, 1971년부터 1975년까지 조지아 지사로 근무했다. 조지아 주지사로 지내면서, 미국에 사는 흑인 등용법을 내세웠다.\n\n대통령 재임 \n\n1976년 미합중국 제39대 대통령 선거에 민주당 후보로 출마하여 도덕주의 정책으로 내세워서, 많은 지지를 받았는데 제럴드 포드 대통령을 누르고 당선되었다.\n\n카터 대통령은 에너지 개발을 촉구했으나 공화당의 반대로 무산되었다.\n\n외교 정책 \n카터는 이집트와 이스라엘을 조정하여 캠프 데이비드에서 안와르 사다트 대통령과 메나헴 베긴 수상과 함께 중동 평화를 위한 캠프데이비드 협정을 체결했다. 이것은 공화당과 미국의 유대인 단체의 반발을 일으켰다. 그러나 1979년, 양국 간의 평화조약이 백악관에서 이루어졌다.\n\n소련과 제2차 전략 무기 제한 협상(SALT II)에 조인했다.\n\n카터는 1970년대 후반 당시 대한민국 등 인권 후진국의 국민들의 인권을 지키기 위해 노력했으

In [15]:
len(data)

100

In [16]:
for rec in data:
    if len(rec['text']) > 35000:
        print('**')

**
**


In [ ]:
# pinecone은 하나의 rec가 40키로바이트를 초과하면 안댐

In [32]:
from langchain_openai import OpenAIEmbeddings
embedding = OpenAIEmbeddings(model='text-embedding-3-small')
embedding

OpenAIEmbeddings(client=<openai.resources.embeddings.Embeddings object at 0x0000026CDD40AB70>, async_client=<openai.resources.embeddings.AsyncEmbeddings object at 0x0000026CDDAE6270>, model='text-embedding-3-small', dimensions=None, deployment='text-embedding-ada-002', openai_api_version=None, openai_api_base=None, openai_api_type=None, openai_proxy=None, embedding_ctx_length=8191, openai_api_key=SecretStr('**********'), openai_organization=None, allowed_special=None, disallowed_special=None, chunk_size=1000, max_retries=2, request_timeout=None, headers=None, tiktoken_enabled=True, tiktoken_model_name=None, show_progress_bar=False, model_kwargs={}, skip_empty=False, default_headers=None, default_query=None, retry_min_seconds=4, retry_max_seconds=20, http_client=None, http_async_client=None, check_embedding_ctx_length=True)

In [ ]:
# 청킹

In [33]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
splitter = RecursiveCharacterTextSplitter(
    chunk_size  = 400,
    chunk_overlap = 20,
    length_function = len, 
    separators=["\n\n","\n"," ",""] # 자르는 순서 문자
)
splitter

In [34]:
import json
def calculate_meatada_size(metadata):
    return len(json.dumps(metadata,ensure_ascii=False).encode('utf-8'))

In [35]:
from uuid import uuid4
import time
texts = []
metas = []
count = 0
batch_size = 20


In [36]:
for i, sample in enumerate(data):
    full_text = sample['text'] # 전문
    # 메타데이터
    metadata = {
        "wiki_id":str(sample['id']),
        "url":sample['url'],
        "title": sample['title'] #[:100]
        }
    
    # 스플릿터로 자르기
    chunks = splitter.split_text(full_text)
    # print (len(chunk))
    
    for i, chunk in enumerate(chunks):
        record = {
            'chunk_id':i,
            'full_text': full_text,
            **metadata
        }
        metadata_size = calculate_meatada_size(record)
        if metadata_size > 35000:
            continue
        texts.append(chunk)
        metas.append(record)
        count += 1
        
        # 청크 배치의 임베딩 -> 적재
        if count % batch_size == 0:
            ids =[str(uuid4()) for _ in range(len(texts))]
            embeddings = embedding.embed_documents(texts) # 20개 청크의 임베딩 수행
            index.upsert(
                vectors=zip(ids, embeddings, metas),
                namespace='wiki-ns1'
            )
            texts=[]
            metas=[]
            time.sleep(1)
     
    

In [ ]:
index.describe_index_stats()

{'dimension': 1536,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'wiki-ns1': {'vector_count': 1160}},
 'total_vector_count': 1160,
 'vector_type': 'dense'}

In [ ]:
# 질의 임베딩

In [39]:
question = ["벨기에는 어디에 있나요?"]
emb_question = embedding.embed_documents(question)

In [40]:
emb_question

[[0.0029317522421479225,
  0.038531601428985596,
  0.0835522934794426,
  0.01172700896859169,
  -0.01034860871732235,
  -0.015395675785839558,
  0.011822436936199665,
  0.015851609408855438,
  -0.028034551069140434,
  -0.02203320525586605,
  -0.019255198538303375,
  0.05802006646990776,
  0.04222147539258003,
  -0.030833764001727104,
  0.0014698522863909602,
  0.0055931261740624905,
  -0.06930174678564072,
  0.01671045832335949,
  0.026719767600297928,
  0.02192717418074608,
  -0.023347988724708557,
  -0.022966276854276657,
  -0.032254576683044434,
  0.048646941781044006,
  0.042560774832963943,
  0.007125271484255791,
  0.01567135564982891,
  0.00029837735928595066,
  0.00684959115460515,
  -0.036156512796878815,
  0.022669389843940735,
  -0.0325726680457592,
  0.021344006061553955,
  -0.05038584768772125,
  0.014197527430951595,
  0.009839660488069057,
  0.045126721262931824,
  -0.0440664105117321,
  0.017866194248199463,
  -0.023560049012303352,
  -0.006059661507606506,
  0.00516370

In [42]:
# 파인콘 질의 실행
q_result = index.query(
    namespace='wiki-ns1',
    vector=emb_question,
    top_k=5,
    include_vector =False,
    include_metadata=True
)

In [43]:
q_result

{'matches': [{'id': '33f9fa3b-3b47-4696-82eb-cd7cf9fa8672',
              'metadata': {'chunk_id': 11.0,
                           'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 '
                                        '없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. '
                                        '통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 '
                                        '로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, '
                                        '열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 '
                                        '루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 '
                                        '라틴 대문자 S를 쓴다.\n'
                                        '\n'
                                        '정의 \n'
                                        '엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 '
                                        '정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. '
                                        '고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 '
                                      

In [45]:
result_ids = [r.id for r in q_result.matches]
result_ids

['33f9fa3b-3b47-4696-82eb-cd7cf9fa8672',
 '1445e91b-803e-41f4-9008-5e3cd5062e80',
 '1f11c2fd-7ad2-428f-9bcc-d674edf50aca',
 '8aeb87d2-e457-4e61-85a0-7fe9b26e88d0',
 'b78bbc90-a865-4508-bd86-398e9ceeebaf']

In [46]:
for r in q_result.matches:
    print(r.score, r.metadata)

0.306918174 {'chunk_id': 11.0, 'full_text': '엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하는데 있어서 점점 신뢰할 수 없는 예측 기술이 되어감으로 인하여 필수적인 것이 되었다.\n\n열역학적 정의 \n고전적 열역학에서는 엔트로피 S의 절대적 값은 정의할 수 없고, 대신 그 상대적 변화만 정의한다. 열적 평형을 이뤄 온도가 인 계에 열 를 가하였다고 하자. 이 경우 엔트로피의 증가

In [ ]:
# 랭체인의 api로 결과 비교하기

In [47]:
question

['벨기에는 어디에 있나요?']

In [48]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[0], k=5, namespace='wiki-ns1')
docs

[Document(id='33f9fa3b-3b47-4696-82eb-cd7cf9fa8672', metadata={'chunk_id': 11.0, 'title': '엔트로피', 'url': 'https://ko.wikipedia.org/wiki/%EC%97%94%ED%8A%B8%EB%A1%9C%ED%94%BC', 'wiki_id': '158'}, page_content='엔트로피(, )는 열역학적 계의 유용하지 않은 (일로 변환할 수 없는) 에너지의 흐름을 설명할 때 이용되는 상태 함수다. 통계역학적으로, 주어진 거시적 상태에 대응하는 미시적 상태의 수의 로그로 생각할 수 있다. 엔트로피는 일반적으로 보존되지 않고, 열역학 제2법칙에 따라 시간에 따라 증가한다. 독일의 물리학자 루돌프 클라우지우스가 1850년대 초에 도입하였다. 대개 기호로 라틴 대문자 S를 쓴다.\n\n정의 \n엔트로피에는 열역학적 정의와 통계학적인 정의, 두 가지의 관련된 정의가 있다. 역사적으로, 고전 열역학적 정의가 먼저 발전하였다. 고전 열역학적인 관점에서, 그 이론은 원자나 분자 같은 수많은 성분들로 이루어져 있고, 학설의 안정성은 그러한 성분들의 평균적인 열특성으로 설명된다.\n이론을 구성하는 성분의 세부적인 성분들은 직접적으로 보이지 않는다. 그러나 그 특성은 온도, 압력, 엔트로피, 열용량과 같은 눈으로 볼 수 있는 평균적인 지표들에 의해 설명된다. 그 이론의 특성에 관한 고전적인 정의는 평형 상태임을 가정하였다. 엔트로피의 고전 열역학적 정의는 최근 비평형 열역학의 영역으로까지 확대되었다. 이후 엔트로피를 포함한 열특성은 눈으로 볼 수 없는 미시세계에서의 움직임에 대한 정역학적인 관점에서 새롭게 정의되었다. 그 예로 처음에는 기체로 여겨졌지만 시간이 지난 후에 광자, 음자, 스핀과 같은 양자 역학적으로 생각되는 뉴턴 입자가 있다. 이 이론의 특성에 대한 통계학적 설명은 고전적인 열역학을 사용하여 이론의 특성을 정의하는 것이 몇몇 변수의 영향을 받는 이론의 최종적인 상태를 예측하

In [ ]:
# 질문의 응답 결과가 동일한지 검증합니다.

In [ ]:
question=[
    "벨기에는 어디에 있나요?",
    "동치관계란 무엇인가요?",
    "멘델레예프는 누구인가요?"
]

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[1], k=5, namespace='wiki-ns1')
docs

[Document(id='2ed9d200-ee23-4d42-811d-621bd1912dee', metadata={'chunk_id': 0.0, 'title': '동치관계', 'url': 'https://ko.wikipedia.org/wiki/%EB%8F%99%EC%B9%98%EA%B4%80%EA%B3%84', 'wiki_id': '154'}, page_content='수학에서 동치관계(同値關係, )는 논리적 동치와 비슷한 성질들을 만족시키는 이항관계이다.\n\n정의 \n집합  위의 동치관계 는 반사관계이자 대칭관계이자 추이관계인 이항관계이다. 즉, 다음 조건들이 성립하여야 한다.\n (반사관계) 임의의 에 대하여, \n (대칭관계) 임의의 에 대하여, 만약 라면, \n (추이관계) 임의의 에 대하여, 만약 이고 라면\n\n동치류와 상집합 \n집합  위에 동치관계 이 주어졌을 때, 원소 의, 동치관계 에 대한 동치류(同値類, ) 는 그 원소와 동치인 원소들의 집합이다 즉,\n\n집합 의 에 대한 상집합(-集合, ) 은 에 대한 동치류들의 집합이다. 즉,\n\n예 \n 임의의 집합 위의 등호 관계\n 도형 집합 위의 닮음 관계\n 사람들의 집합 위의, 같은 생일을 갖는 관계\n\n반례 \n 공집합이 아닌 집합  위의 공관계는 (유일한 유형의) 반사관계가 아닌 대칭관계이자 추이관계이다.\n 실수 집합 위의 순서 관계 는 대칭관계가 아닌 반사관계이자 추이관계이다.\n 실수 집합 위의 이항관계 은 추이관계가 아닌 반사관계이자 대칭관계이다.\n\n성질\n\n표준 사상 \n집합 위의 동치관계로부터, 표준사상을 구성할 수 있다. 즉, 집합  위의 동치관계 에 대하여, 함수\n\n를 표준사상이라고 한다.\n\n반대로, 전사함수 에 대하여, 이항관계\n\n는 동치관계이며, 그 상집합은\n\n이다.\n\n집합의 분할 \n집합 위의 동치관계와 그 집합의 분할 사이에는 자연적인 일대일 대응이 존재한다. 즉, 다음과 같다.\n\n집합  위의 동치관계 에 대하여, 그 상집합 은 의 분할

In [52]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding=embedding, text_key='full_text')
docs = vector_store.similarity_search(query=question[2], k=5, namespace='wiki-ns1')
docs

[Document(id='ce3942fb-960f-45e4-ac0e-b9b57e8a18f1', metadata={'chunk_id': 7.0, 'title': '주기율표', 'url': 'https://ko.wikipedia.org/wiki/%EC%A3%BC%EA%B8%B0%EC%9C%A8%ED%91%9C', 'wiki_id': '76'}, page_content='주기율표(週期律表, , ) 또는 주기표(週期表)는 원소를 구분하기 쉽게 성질에 따라 배열한 표로, 러시아의 드미트리 멘델레예프가 처음 제안했다. 1915년 헨리 모즐리는 멘델레예프의 주기율표를 개량시켜서 원자번호순으로 배열했는데, 이는 현대의 원소 주기율표와 유사하다. 원자 번호가 커짐에 따라 성질이 비슷한 원소가 주기적으로 나타나는 성질인 주기성을 기준으로 원소들을 배열하였다. 주기율표의 가로행은 주기라 부르고, 세로열은 족이라 부른다. 주기마다 같은 성질의 원소가 반복적으로 나타나기 때문에, 같은 족의 원소들은 서로 유사한 화학적 특성을 보인다. 전자를 가지고 있으려 하는 비금속성은 대체로 오른쪽이 더 높으며, 반대로 전자를 주려고 하는 금속성은 대체로 왼쪽이 더 높다. 이러한 화학적 성질은 각 원소의 전자 배치에 기인한다.\n\n1869년 러시아의 화학자 드미트리 멘델레예프가 원자 질량에 따라 원소의 화학적 성질이 주기적으로 변화하는 것에 착안하여 주기율표를 처음으로 만들었다. 당시에는 모든 원소가 발견되지 않았기 때문에 원소 사이에 공백이 남아있었는데, 멘델레예프는 원소의 주기성에 착안하여 원소를 새로 발견하기도 하였다. 원소의 주기성은 19세기 후반에 사실로 인식되었으며, 원자 번호가 발견되고 20세기 초에 양자역학을 통해 원자의 내부 구조를 탐구하며 재확인되었다. 글렌 T. 시보그가 1945년에 악티늄족이 d-블록 원소가 아닌 f-블록 원소라는 사실을 발견함으로써 현대의 주기율표 틀이 완성되었다.\n\n주기율표는 과학의 발전에 따라 계속 개정되고 있다. 자연계에서는 원자 번호 94까지 존

In [53]:
# retrieval
country ='벨기에'
query = f"{country}의 여행을 위한 관광지와 문화"
docs = vector_store.similarity_search(query=query, k=5, namespace='wiki-ns1')

In [ ]:
# Augmentation context

In [54]:
context = ""
for result in docs:
    context += result.page_content

In [55]:
# generation
from langchain_openai import ChatOpenAI
llm = ChatOpenAI(model="gpt-4o-mini")
llm

ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x0000026CDDAE64B0>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x0000026CDD409E20>, root_client=<openai.OpenAI object at 0x0000026CDDAA1DF0>, root_async_client=<openai.AsyncOpenAI object at 0x0000026CDDAA2030>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********'))

In [56]:
from langchain.prompts import ChatPromptTemplate

chat_template = ChatPromptTemplate.from_messages(
    [
        ('system',"""당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행일정 수립을 도와드립니다. 제한된 정보라도 치대한 활용해서 도움을 주세요."""),
        ('human',"""저는 {country}로 여행을 계획중입니다. 아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요. 만약, 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.
         context :
         {context}"""),
    ]
)
chat_template


ChatPromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행일정 수립을 도와드립니다. 제한된 정보라도 치대한 활용해서 도움을 주세요.'), additional_kwargs={}), HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['context', 'country'], input_types={}, partial_variables={}, template='저는 {country}로 여행을 계획중입니다. 아래 context에서 {country}관련 정보를 찾아 여행 조언을 해주세요. 만약, 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.\n         context :\n         {context}'), additional_kwargs={})])

In [57]:
prompt = chat_template.format_messages(country=country, context=context)
print(prompt)

[SystemMessage(content='당신은 베테랑 여행 가이드입니다. 고객 맞춤 여행일정 수립을 도와드립니다. 제한된 정보라도 치대한 활용해서 도움을 주세요.', additional_kwargs={}, response_metadata={}), HumanMessage(content='저는 벨기에로 여행을 계획중입니다. 아래 context에서 벨기에관련 정보를 찾아 여행 조언을 해주세요. 만약, 직접적인 정보가 없다면, 수도나 관련 정보를 활용해 주세요.\n         context :\n         아쿠타가와 류노스케(, 1892년 3월 1일~1927년 7월 24일)는 일본의 소설가이다. 호는 징강당주인(澄江堂主人)이며 하이쿠 작가로서의 호는 가키(我鬼)이다.\n\n그의 작품은 대부분이 단편 소설이다. 「참마죽」, 「덤불 속」, 「지옥변」 등 주로 일본의 《곤자쿠 이야기집》·《우지슈이 이야기》 등 전통적인 고전들에서 제재를 취하였다. 또한 「거미줄(원제: 蜘蛛の糸)」, 「두자춘(杜子春)」 등 어린이를 위한 작품도 남겼으며, 예수를 학대한 유대인이 예수가 세상에 다시 올 때까지 방황한다는 상상력을 발휘한 「방황하는 유대인」도 있다.\n\n생애\n\n유년 시절 \n1892년(메이지 25년) 3월 1일 도쿄에서 우유 판매업자였던 아버지 니하라 도시조(新原敏三)와 어머니 후쿠(フク) 사이의 아들로 태어났다(아쿠타가와라는 성은 원래 그의 어머니쪽 성씨였다). 이때 태어난 시간이 공교롭게도 진년(辰年) 진월(辰月) 진일(辰日) 진시(辰時)였기 때문에 \'용(龍)\' 자를 이름에 넣어 류노스케(龍之介)라 짓게 되었다고 전하나, 실제 그가 태어난 1892년 3월 1일은 간지로는 임진년·임인월·임진일에 해당하며, 출생 시각에 대해서는 자료가 없기 때문에 확실한 것이 없다. 이름도 호적상으로는 \'龍之介\'이지만 그가 양자로 들어갔던 아쿠타가와 집안이나 졸업한 학교의 명단 등의 문서에는 \'龍之助\'로 되어 있다(아쿠타가와 자신은 \'龍之助\' 표기를 싫어했다).\

In [58]:
response = llm.invoke(prompt)
response.content

'벨기에 여행은 다양한 문화와 역사, 그리고 맛있는 음식으로 가득한 경험이 될 것입니다. 아래는 벨기에 여행을 위한 몇 가지 유용한 조언입니다.\n\n### 주요 도시와 명소\n1. **브뤼셀 (Brussels)**:\n   - **그랑 플라스 (Grand Place)**: UNESCO 세계유산으로 등록된 아름다운 광장입니다. 주변의 건축물들은 모두 인상적입니다.\n   - **아토미움 (Atomium)**: 1958년 브뤼셀 세계 박람회를 기념하기 위해 세운 건축물입니다. 독특한 디자인과 전망대에서의 멋진 경치를 즐길 수 있습니다.\n\n2. **브루게 (Bruges)**:\n   - **중세 도시**: 운하와 중세 건축물로 유명한 이 도시는 마치 동화 속에 들어간 듯한 느낌을 줍니다. 자전거를 타고 돌아다니기 좋습니다.\n   - **바르로드 카페 (Belfry Tower)**: 이곳의 탑에 올라가면 도시 전경을 한눈에 볼 수 있습니다. \n\n3. **안트워프 (Antwerp)**:\n   - **안트워프 성당 (Cathedral of Our Lady)**: 유명한 예술작품들이 있는 성당이며, 특히 루벤스의 작품을 감상할 수 있는 기회가 있습니다.\n   - **다이아몬드 지구**: 보고 싶다면 다이아몬드 상점들이 즐비한 이곳을 발길에 따라 구경할 수 있습니다.\n\n### 음식과 음료\n- **벨기에 와플**: 브뤼셀 스타일과 리에주 스타일 두 가지가 있으며, 다양한 토핑을 통해 즐길 수 있습니다.\n- **맥주**: 벨기에는 다양한 종류의 맥주가 있으며, 많은 양조장이 있습니다. 각각의 맥주에 맞는 잔도 고유합니다.\n- **초콜릿**: 벨기에 초콜릿은 세계적으로 유명합니다. 직접 만들어보는 워크숍도 즐길 수 있으니, 관심 있는 분은 검색해 보세요.\n\n### 교통\n- **기차**: 벨기에는 기차망이 잘 구축되어 있어 주요 도시 간 이동이 매우 편리합니다.\n- **자전거**: 브뤼셀과 브루게는 자전거 타기에 좋은 도로가 잘 되어 있어, 자전거를 

In [59]:
# 벡터 검색 + 키워드 필터링
def enhanced_search(country, vector_store):
    # 1단계: 벡터 검색
    vector_results = vector_store.similarity_search(
        query=f"{country} 여행 관광", 
        k=10, 
        namespace="wiki-ns1"
    )
    
    # 2단계: 국가명이 포함된 결과 우선순위
    country_related = []
    other_results = []
    
    for result in vector_results:
        if country.lower() in result.page_content.lower() or \
           "벨기에" in result.page_content or \
           "브뤼셀" in result.page_content:
            country_related.append(result)
        else:
            other_results.append(result)
    
    # 관련성 높은 결과 우선 반환
    final_results = (country_related + other_results)[:5]
    return final_results

results = enhanced_search("벨기에", vector_store)

In [60]:
results

[Document(id='b78bbc90-a865-4508-bd86-398e9ceeebaf', metadata={'chunk_id': 5.0, 'title': '나라 이름순 수도 목록', 'url': 'https://ko.wikipedia.org/wiki/%EB%82%98%EB%9D%BC%20%EC%9D%B4%EB%A6%84%EC%88%9C%20%EC%88%98%EB%8F%84%20%EB%AA%A9%EB%A1%9D', 'wiki_id': '79'}, page_content='국\n\nㄱ \n\n 가나 - 아크라 \n 가봉 - 리브르빌\n 가이아나 - 조지타운\n 감비아 - 반줄\n 괌 - 하갓냐\n 과테말라 - 과테말라\n 그레나다 - 세인트조지스\n 그리스 - 아테네\n 그린랜드 - 누크\n 기니 - 코나크리\n 기니비사우 - 비사우\n\nㄴ \n\n 나미비아 - 빈트후크\n 나우루 - 야렌\n 나이지리아 - 아부자\n 남수단 - 주바\n 남아프리카 공화국 - 프리토리아(행정), 블룸폰테인(사법), 케이프타운(입법)\n 남오세티야 - 츠힌발리\n 네덜란드 - 암스테르담(헌법 상), 헤이그(행정)\n 네팔 - 카트만두\n 노르웨이 - 오슬로\n 뉴질랜드 - 웰링턴\n 니우에 - 알로피\n 니제르 - 니아메\n 니카라과 - 마나과\n\nㄷ \n\n 대한민국 - 서울특별시(사실상), 세종특별자치시(행정)\n 덴마크 - 코펜하겐\n 도미니카 연방 - 로조\n 도미니카 공화국 - 산토도밍고\n 독일 - 베를린\n 동티모르 - 딜리\n\nㄹ \n\n 라오스 - 비엔티안\n 라이베리아 - 몬로비아\n 라트비아 - 리가\n 러시아 - 모스크바\n 레바논 - 베이루트\n 레소토 - 마세루\n 루마니아 - 부쿠레슈티\n 룩셈부르크 - 룩셈부르크\n 르완다 - 키갈리\n 리비아 - 트리폴리\n 리투아니아 - 빌뉴스\n 리히텐슈타인 - 파두츠\n\nㅁ \n\n 마다가스카르 - 안타나나리보\n 마셜 제도 - 마주로\n 마케도니아 공화국 - 스코페\n 말라위 - 릴롱궤\n 말레이시아